In [1]:
import netCDF4 as nc
import pandas as pd
from matplotlib import pyplot as plt
import xarray as xr
import numpy as np
import scipy
import plotly.express as px
from scipy.stats import mstats
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import math
import csv
import fiona
import geopandas
pd.set_option('display.max_rows', 24000)

In [ ]:
dat_dir = f'/projects/kopp/dtb96/branches/Mortality_Project/data/2_projections/3_impacts/main_specification'
data_dict = {'fn_full: '',
             'fn_inc: '',
             'fn_none: '',
             'fn_histclem:'', 
            }
rcps = ['45','85']
uncertainty_ranges = ['low','high']

path = f'{dat_dir}/{rcp}/CCSM4/{uncertainty_range}/SSP3/{data_dict[this_file]}'
    


In [2]:
#loading in mortality dataset from hierarchy file (available on Zenodo)
#fn_full_agg is the dataset with full adaptation

fn_full = "/projects/kopp/dtb96/branches/Mortality_Project/data/2_projection/3_impacts/main_specification/raw/single/rcp85/CCSM4/low/SSP3/Agespec_interaction_GMFD_POLY-4_TINV_CYA_NW_w1-oldest.nc4"

#fn_inc is the dataset with income adaptation only
fn_inc = "/projects/kopp/dtb96/branches/Mortality_Project/data/2_projection/3_impacts/main_specification/raw/single/rcp85/CCSM4/low/SSP3/Agespec_interaction_GMFD_POLY-4_TINV_CYA_NW_w1-oldest-incadapt.nc4"

#fn_none is the dataset with no adaptation, only temperature change

fn_none = "/projects/kopp/dtb96/branches/Mortality_Project/data/2_projection/3_impacts/main_specification/raw/single/rcp85/CCSM4/low/SSP3/Agespec_interaction_GMFD_POLY-4_TINV_CYA_NW_w1-oldest-noadapt.nc4"

#fn_histclim_agg is the HistClim dataset
fn_histclim = "/projects/kopp/dtb96/branches/Mortality_Project/data/2_projection/3_impacts/main_specification/raw/single/rcp85/CCSM4/low/SSP3/Agespec_interaction_GMFD_POLY-4_TINV_CYA_NW_w1-oldest-histclim.nc4"


In [3]:
#loading in shapefile associated with hierarchy file
#shapefile_5k is the aggregated dataset binned by 5,678 regions
shapefile_5k = "/projects/kopp/dtb96/branches/Mortality_Project/data/impact_high_res"

#shapefile_24k is the full resolution dataset binned by 24,378 regions
shapefile_24k = "/projects/kopp/dtb96/branches/Mortality_Project/data/new_shapefile_nytimes"

#reading in the shapefiles with geopandas
sf_5k = geopandas.read_file(shapefile_5k)
sf_24k = geopandas.read_file(shapefile_24k)


In [4]:
#Cleaning raw mortality Datasets so region dimension is helpful. Thanks Brewster!
def fix_region(ds):
    

    #Changes coords/dims/vars so 'region' is region names we can select data by.
    #For e.g. `ds.sel(region='IND.10.121')` should work properly.
  
    return ds.set_coords("regions").swap_dims({"region": "regions"}).rename({"regions": "region"})


ds_full = fix_region(xr.open_dataset(fn_full))
ds_inc = fix_region(xr.open_dataset(fn_inc))
ds_none = fix_region(xr.open_dataset(fn_none))
ds_histclim = fix_region(xr.open_dataset(fn_histclim))

In [5]:
ds_full

<xarray.Dataset>
Dimensions:            (year: 120, region: 24378, operation: 4)
Coordinates:
  * year               (year) int32 1981 1982 1983 1984 ... 2097 2098 2099 2100
  * region             (region) object 'CAN.1.2.28' ... 'BWA.4.13'
Dimensions without coordinates: operation
Data variables:
    rebased            (year, region) float32 ...
    transformed        (year, region) float32 ...
    response           (year, region) float32 ...
    climtas_effect     (year, region) float32 ...
    orderofoperations  (operation) object ...
Attributes:
    description:   Mortality impacts, with interpolation and adaptation throu...
    version:       Agespec_interaction_GMFD_POLY-4_TINV_CYA_NW_w1-oldest.2019...
    dependencies:  1.6, Hierarchy.2016-02-03, Hierarchy.2016-02-03, GCP-GDPPC...
    author:        James Rising

In [6]:
#test = ds['region'].to_pandas()
#display(test.to_string())

In [7]:
# #isolating individual regions to analyze - using this for meshgrid

# DEL = ds.sel(region ='IND.10.121.371')    #Delhi - 3 versions listed in hierarchy file
# OSL = ds.sel(region ='NOR.12.288')        #Oslo
# CHI = ds.sel(region ='USA.24.1332')       #Cook County aka Chicagoland area
# SPB = ds.sel(region ='BRA.25.5269.9934')  #Fatima Paulista, had problems with Sao Paulo ID
# ACC = ds.sel(region ='GHA.5.70')          #Accra
# SYD = ds.sel(region ='AUS.4.242')         #North Sydney 
# BEI = ds.sel(region ='CHN.2.18.84')       #Miyun District - had problems with Beijing ID

# #isolating the regions again for the difference calculation down the road
# DEL1 = ds1.sel(region ='IND.10.121.371')
# OSL1 = ds1.sel(region ='NOR.12.288')
# CHI1 = ds1.sel(region ='USA.24.1332')        
# SPB1 = ds1.sel(region ='BRA.25.5269.9934')  
# ACC1 = ds1.sel(region ='GHA.5.70')
# SYD1 = ds1.sel(region ='AUS.4.242')    
# BEI1 = ds1.sel(region ='CHN.2.18.84')


In [8]:
#taking the 20yr rolling quantile values from the 'rebased' dataset
q = (0.17, 0.5, 0.83)
window_n = 20
rebased_qs_full = ds_full["rebased"].rolling({"year": window_n}, center=True).construct("tmp").quantile(q,skipna= False, dim="tmp")
rebased_qs_inc = ds_inc["rebased"].rolling({"year": window_n}, center=True).construct("tmp").quantile(q,skipna= False, dim="tmp")
rebased_qs_none = ds_none["rebased"].rolling({"year": window_n}, center=True).construct("tmp").quantile(q,skipna= False, dim="tmp")
rebased_qs_histclim = ds_histclim["rebased"].rolling({"year": window_n}, center=True).construct("tmp").quantile(q,skipna= False, dim="tmp")



In [9]:
# a_1 = np.where(rebased_qs.sel(region="AUS.4.242", quantile=0.17, year 2010) >=rebased_qs.sel(region="AUS.4.242",quantile=0.83, year =2010))
# a_1 = np.asarray(a_1)
# a1 = a_1.min(axis=0)
# amin1 = min(a1)

# TOE_labela1 = amin1 + 1981

# rebased_qs.sel(region="AUS.4.242").plot.line(x="year")
# plt.title("AUS.4.242/CCSM4/low/SSP3/oldest/no-adapt")
# plt.axvline(x = amin1+1981, linestyle='--',c='black' ,label = "TOE - " + str(TOE_labela1) )
# plt.axhline(y=rebased_qs.sel(region="AUS.4.242", quantile=0.17,year=2010), linestyle='--', c='b')
# plt.axhline(y=rebased_qs.sel(region="AUS.4.242", quantile=0.83,year=2010), linestyle='--', c='g')
# plt.legend()

In [10]:
# a_2 = np.where(rebased_qs1.sel(region="AUS.4.242", quantile=0.17) >=rebased_qs1.sel(region="AUS.4.242",quantile=0.83, year =2010))
# a_2 = np.asarray(a_2)
# a2 = a_2.min(axis=0)
# amin2 = min(a2)

# TOE_labela2 = amin2 + 1981

# rebased_qs1.sel(region="AUS.4.242").plot.line(x="year")
# plt.title("AUS.4.242/CCSM4/low/SSP3/oldest/inc-adapt")
# plt.axvline(x = amin2+1981, linestyle='--',c='black' ,label = "TOE - " + str(TOE_labela) )
# plt.axhline(y=rebased_qs1.sel(region="AUS.4.242", quantile=0.83,year=2010), linestyle='--', c='g')
# plt.legend()

In [11]:
#Simple difference between full income-full adaptation and Histclim
difference = ds_full["rebased"] - ds_histclim["rebased"]

rebased_difference = difference.rolling({"year": window_n}, center=False).construct("tmp").quantile(q,skipna= False, dim="tmp")



In [12]:
#finding where 83rd %-tile crosses the 17th %-tile, or vice versa
window = slice(2000,2100)

def find_quants(data_path,quants,yr):
    data = xr.open(data_path)
    threshold = data.sel(quantile=quants,year=yr)
    return threshold
    

def fun_threshold_full(quant,yr):
    threshold_full = rebased_qs_full.sel(quantile=quant, year=yr)
    return threshold_full

def fun_threshold_inc(quant,yr):
    threshold_inc = rebased_qs_inc.sel(quantile=quant, year=yr)
    return threshold_inc

def fun_threshold_none(quant,yr):
    threshold_none = rebased_qs_none.sel(quantile=quant, year=yr)
    return threshold_none

def fun_threshold_histclim(quant,yr):
    threshold_histclim = rebased_qs_histclim.sel(quantile=quant, year=yr)
    return threshold_histclim

def fun_threshold_difference(quant,yr):
    threshold_histclim = rebased_qs_histclim.sel(quantile=quant, year=yr)
    return threshold_histclim

In [13]:
#benefits earliest threshold crossing - this gives TOE
#for full adapt
mask_years_over_threshold_benefits_full = fun_threshold_full(0.17, 2000) >= fun_threshold_full(0.83, window)
earliest_threshold_year_benefits_full = mask_years_over_threshold_benefits_full.idxmax(dim="year")

#for income adapt
mask_years_over_threshold_benefits_inc = fun_threshold_inc(0.17, 2000) >= fun_threshold_inc(0.83, window)
earliest_threshold_year_benefits_inc = mask_years_over_threshold_benefits_inc.idxmax(dim="year")

#for no adapt
mask_years_over_threshold_benefits_none = fun_threshold_none(0.17, 2000) >= fun_threshold_none(0.83, window)
earliest_threshold_year_benefits_none = mask_years_over_threshold_benefits_none.idxmax(dim="year")

#for histclim
mask_years_over_threshold_benefits_histclim = fun_threshold_histclim(0.17, 2000) >= fun_threshold_histclim(0.83, window)
earliest_threshold_year_benefits_histclim = mask_years_over_threshold_benefits_histclim.idxmax(dim="year")

#for the difference between full adaptation and histclim
mask_years_over_threshold_benefits_difference = fun_threshold_difference(0.17, 2000) >= fun_threshold_difference(0.83, window)
earliest_threshold_year_benefits_difference= mask_years_over_threshold_benefits_difference.idxmax(dim="year")


In [14]:
#damages earliest threshold crossing - this gives TOE
#for full adapt
mask_years_over_threshold_damages_full = fun_threshold_full(0.83, 2000) <= fun_threshold_full(0.17, window)
earliest_threshold_year_damages_full = mask_years_over_threshold_damages_full.idxmax(dim="year")

#for income adapt
mask_years_over_threshold_damages_inc = fun_threshold_inc(0.83, 2000) <= fun_threshold_inc(0.17, window)
earliest_threshold_year_damages_inc = mask_years_over_threshold_damages_inc.idxmax(dim="year")

#for no adapt
mask_years_over_threshold_damages_none = fun_threshold_none(0.83, 2000) <= fun_threshold_none(0.17, window)
earliest_threshold_year_damages_none = mask_years_over_threshold_damages_none.idxmax(dim="year")

#for histclim
mask_years_over_threshold_damages_histclim = fun_threshold_histclim(0.83, 2000) <= fun_threshold_histclim(0.17, window)
earliest_threshold_year_damages_histclim = mask_years_over_threshold_damages_histclim.idxmax(dim="year")

#for the difference between full adapt and histclim
mask_years_over_threshold_damages_difference = fun_threshold_difference(0.83, 2000) <= fun_threshold_difference(0.17, window)
earliest_threshold_year_damages_difference = mask_years_over_threshold_damages_difference.idxmax(dim="year")


In [15]:
earliest_threshold_year_benefits_full

<xarray.DataArray 'year' (region: 24378)>
array([2024, 2024, 2032, ..., 2025, 2029, 2029], dtype=int32)
Coordinates:
  * region   (region) object 'CAN.1.2.28' 'CAN.1.17.403' ... 'BWA.5' 'BWA.4.13'

In [16]:
sf_24k

,gadmid,hierid,color,ISO,geometry
0,28115,CAN.1.2.28,1,CAN,"POLYGON ((-110.05459 53.30730, -110.00587 53.3..."
1,28116,CAN.1.17.403,2,CAN,"POLYGON ((-111.23055 52.91943, -111.22076 52.9..."
2,28119,CAN.2.34.951,3,CAN,"POLYGON ((-127.68527 55.29570, -127.69742 55.3..."
3,28120,CAN.11.259.4274,4,CAN,"POLYGON ((-77.73080 55.31879, -77.71030 55.317..."
4,28124,CAN.11.269.4448,5,CAN,"POLYGON ((-66.25940 54.99975, -65.24999 55.000..."
...,...,...,...,...,...
24373,6905,BWA.1,24374,BWA,"POLYGON ((26.01723 -23.55865, 25.86139 -23.355..."
24374,6902,BWA.7,24375,BWA,"POLYGON ((22.50413 -18.11382, 22.98841 -18.018..."
24375,6915,BWA.6.16,24376,BWA,"POLYGON ((27.28497 -20.49781, 27.35821 -20.473..."
24376,6909,BWA.5,24377,BWA,"POLYGON ((23.05667 -23.31139, 23.55222 -23.313..."


In [17]:
#adding TOE of benefits or damages to the shapefiles and removing NaN values
#for full adapt
sf_24k['TOE_benefits_full'] = earliest_threshold_year_benefits_full
sf_24k['TOE_damages_full'] = earliest_threshold_year_damages_full
sf_24k['TOE_benefits_full'] = sf_24k['TOE_benefits_full'].replace(2010,np.NaN)
sf_24k['TOE_damages_full'] = sf_24k['TOE_damages_full'].replace(2010,np.NaN)

#for income adapt
sf_24k['TOE_benefits_inc'] = earliest_threshold_year_benefits_inc
sf_24k['TOE_damages_inc'] = earliest_threshold_year_damages_inc
sf_24k['TOE_benefits_inc'] = sf_24k['TOE_benefits_inc'].replace(2010,np.NaN)
sf_24k['TOE_damages_inc'] = sf_24k['TOE_damages_inc'].replace(2010,np.NaN)

#for no adapt
sf_24k['TOE_benefits_none'] = earliest_threshold_year_benefits_none
sf_24k['TOE_damages_none'] = earliest_threshold_year_damages_none
sf_24k['TOE_benefits_none'] = sf_24k['TOE_benefits_none'].replace(2010,np.NaN)
sf_24k['TOE_damages_none'] = sf_24k['TOE_damages_none'].replace(2010,np.NaN)

#for histclim
sf_24k['TOE_benefits_histclim'] = earliest_threshold_year_benefits_histclim
sf_24k['TOE_damages_histclim'] = earliest_threshold_year_damages_histclim
sf_24k['TOE_benefits_histclim'] = sf_24k['TOE_benefits_histclim'].replace(2010,np.NaN)
sf_24k['TOE_damages_histclim'] = sf_24k['TOE_damages_histclim'].replace(2010,np.NaN)

#for the difference between full adapt and histclim
sf_24k['TOE_benefits_difference'] = earliest_threshold_year_benefits_difference
sf_24k['TOE_damages_difference'] = earliest_threshold_year_damages_difference
sf_24k['TOE_benefits_difference'] = sf_24k['TOE_benefits_difference'].replace(2010,np.NaN)
sf_24k['TOE_damages_difference'] = sf_24k['TOE_damages_difference'].replace(2010,np.NaN)


In [18]:
type(sf_24k)

geopandas.geodataframe.GeoDataFrame

In [19]:
econ = "/projects/kopp/dtb96/branches/Mortality_Project/data/2_projection/2_econ_vars/SSP3.nc4"
econ_vars = xr.open_dataset(econ)
econ_vars = econ_vars

econ_vars['gdp'][1]

#pull region names, make array, and loop through the names, not the index (perhaps sort alpha. and use regions names)
regions = 

<xarray.DataArray 'gdp' (region: 24378, year: 91)>
[2218398 values with dtype=float64]
Coordinates:
    ssp      object ...
    model    <U3 'low'
  * region   (region) object 'ABW' 'AFG.1.12' ... 'ZWE.9.R19a261b63a503c9c'
  * year     (year) int64 2010 2011 2012 2013 2014 ... 2096 2097 2098 2099 2100

In [21]:
#take 2050 and 2090 and take raw data for the same years
gdp = econ_vars['gdp'][1].rolling({"year": window_n}, center=False).mean()
pop65plus = econ_vars['pop65plus'][1].rolling({"year": window_n}, center=True).mean()




In [92]:
pop65plus_2050 = pop65plus.sel(year = 2050)
pop65plus_2090 = pop65plus.sel(year = 2090)

gdp_2050 = pd.DataFrame(gdp.sel(year = 2050), columns = ['hierid','gdp_2050'])
gdp_2090 = gdp.sel(year = 2090)

#gdp_2050 = gdp_2050.rename({'region': 'hierid'}).to_pandas()

#pop65plus = pop65plus.rename({'region': 'hierid'}).to_pandas()

#sf_24k['gdp_2050'] = sf_24k.merge(gdp_2050,on='hierid')

#sf_24k

gdp_2050

ValueError: Shape of passed values is (24378, 1), indices imply (24378, 2)

In [ ]:
#sf_24k['gdp_2050'] = gdp[:,40]
sf_24k['gdp_2090'] = gdp.sel(year = 2090)
sf_24k

In [ ]:
# # Function to calculate meshgrid diff between full aggregated data and HistClim

# def fn_diff(Xvar,Yvar,step,quantile):
#     tot_len=len(Xvar[0])
#     diff=[]
#     for ii in range(tot_len-step):
#         if ii == (tot_len):
#             break #break loop if it exceeds window.
#         else:
#             diff_var=Xvar[0+ii:ii+step,0+ii:ii+step] - Yvar[0+ii:ii+step,0+ii:ii+step]
#             quant_diff = np.quantile(diff_var, quantile)
#             diff.append(quant_diff)
#     return diff



In [ ]:
# #creating np.meshgrid dataframe between full income/full adapt. and Histclim
# p = xx,yy = np.meshgrid(DEL['rebased'],DEL1['rebased'])
# step=30

# diff17 = fn_diff(xx,yy,30,.17)
# diff50 = fn_diff(xx,yy,30,.50)
# diff83 = fn_diff(xx,yy,30,.83)


# #stack results all at once
# Diff17 = np.vstack(diff17)
# Diff50 = np.vstack(diff50)
# Diff83 = np.vstack(diff83)

# #creating dataframe to store stacked results
# data = pd.DataFrame()


# data['17th_percentile'] = Diff17.flatten()
# data['50th_percentile'] = Diff50.flatten()
# data['83rd_percentile'] = Diff83.flatten()


# data = data.reindex(data.index.tolist() + list(range(30))).shift(30)
# data = data.reset_index()
# data = data.iloc[:120]


In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sf_24k.plot(column="ISO",ax=ax,legend=False, color='grey')
sf_24k.plot(column="TOE_benefits_full",ax=ax,legend=True)
plt.title('TOE of Benefits - CCSM4/low/SSP3/oldest/full-adapt')


In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sf_24k.plot(column="ISO",ax=ax,legend=False, color='grey')
sf_24k.plot(column="TOE_benefits_inc",ax=ax,legend=True)
plt.title('TOE of Benefits - CCSM4/low/SSP3/oldest/inc-adapt')


In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sf_24k.plot(column="ISO",ax=ax,legend=False, color='grey')
sf_24k.plot(column="TOE_benefits_none",ax=ax,legend=True)
plt.title('TOE of Benefits - CCSM4/low/SSP3/oldest/no-adapt')


In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sf_24k.plot(column="ISO",ax=ax,legend=False, color='grey')
sf_24k.plot(column="TOE_benefits_histclim",ax=ax,legend=True)
plt.title('TOE of Benefits - CCSM4/low/SSP3/oldest/histclim')


In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sf_24k.plot(column="ISO",ax=ax,legend=False, color='grey')
sf_24k.plot(column="TOE_benefits_difference",ax=ax,legend=True)
plt.title('TOE of Benefits - CCSM4/low/SSP3/oldest/difference')


In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sf_24k.plot(column="ISO",ax=ax,legend=False, color='grey')
sf_24k.plot(column="gdp",ax=ax,legend=True)
plt.title('TOE of Benefits - CCSM4/low/SSP3/oldest/difference')
